## 초기 라이브러리 및 함수 설정

In [ ]:

import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

ENV_NAME = "CartPole-v1" # Game 이름 

GAMMA = 0.95 # Discount Factor
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000 #Replay memory최대 저장 개수
BATCH_SIZE = 20 #memory에 history data가 20개까지 있을 때

EXPLORATION_MAX = 1.0 #최대 exploration 값 
EXPLORATION_MIN = 0.01 #최소 exploration 값
EXPLORATION_DECAY = 0.995 #exploration 값이 작아지게 하기 위해서 



## DQN network 구현

In [16]:

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX #초반에 Exploration으로 탐색을 하는 비율정의

        self.action_space = action_space # Action Space :2
        self.memory = deque(maxlen=MEMORY_SIZE) # Replay Memory 정의: deque함수로 정의  
        
        ## Feed Foward Network 정의 
        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE)) # loss function 및 optimizer 정의

    


## Replay memory 저의

In [11]:
    def remember(self, state, action, reward, next_state, done):
        # agent의 state, action, reward, next_state, 게임이 끝났는지 등을 메모리 버퍼에 저장
        self.memory.append((state, action, reward, next_state, done))

    

## Action 함수 정의

In [12]:
    def act(self, state):
        #exploration_rate가 초반에는 1.0 이지만, 학습을 거급하면서 점점 작아진다.
        if np.random.rand() < self.exploration_rate:
            # Random으로 action을 선택
            return random.randrange(self.action_space)
        # agent가 직접 action을 선택
        q_values = self.model.predict(state)
        # 오른쪽, 왼쪽으로 예측한 값 중에 확률값이 높은것을 선택해서 return 한다
        return np.argmax(q_values[0])

## 학습 Process 정의

In [1]:
    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)# 버퍼에서 batch size만큼 rand
        for state, action, reward, state_next, terminal in batch:
            q_update = reward #현제  state와 action을 했을 때 reward
            if not terminal:# 이 Action을 했을 때 게임이 끝나지 않았다면
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0])) # q_update -> reward를 계산 
            q_values = self.model.predict(state)
            q_values[0][action] = q_update # 기존 action의 q_value값 다시 계산 : 게임이 끝났다면 기존 reward를 끝나지 않았다면 예측한 q_update 값
            self.model.fit(state, q_values, verbose=0) #model 학습 
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)#exploration_rate 줄여나감

## Main 실행 함수